In [1058]:
from urllib.request import urlopen
import pandas as pd
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup

In [1059]:
def trata_html(input):
    return " ".join(html.split()).replace('> <', '><')

def gera_dicionario(lista, ano):
    dicionario = {'Time' : [], 'Jogadores': [], 'Média de Idade': [], 'Estrangeiros': []\
               , 'Média de valor de mercado': [], 'Valor de mercado total': [], 'Ano' : []}
    i = 0
    for item in lista:
        i += 1
        if i > 6:
            i = 1

        if i == 1:
            dicionario['Time'].append(item)
        elif i == 2:
            dicionario['Jogadores'].append(item)
        elif i == 3:
            dicionario['Média de Idade'].append(item)
        elif i == 4:
            dicionario['Estrangeiros'].append(item)
        elif i == 5:
            dicionario['Média de valor de mercado'].append(item)
        elif i == 6:
            dicionario['Valor de mercado total'].append(item)
            dicionario['Ano'].append(ano)
    return dicionario

In [1060]:
adaptacao = {
    'São Paulo FC' : 'São Paulo',
    'SC Internacional' : 'Internacional',
    'SC Corinthians' : 'Corinthians',
    'Fluminense FC' : 'Fluminense',
    'Botafogo FR' : 'Botafogo',
    'Grêmio FBPA' : 'Grêmio',
    'CR Flamengo' : 'Flamengo',
    'Cruzeiro EC' : 'Cruzeiro',
    'Atlético Mineiro' : 'Atlético-MG',
    'Clube Atlético Paranaense' : 'Athletico-PR','Athletico Paranaense' : 'Athletico-PR', 'Atlético-PR' : 'Athletico-PR',
    'Paraná Clube' : 'Paraná',
    'Goiás EC' : 'Goiás',
    'CR Vasco da Gama' : 'Vasco da Gama',
    'América FC (RN)' : 'América-RN',
    'EC Juventude' : 'Juventude',
    'SE Palmeiras' : 'Palmeiras',
    'Santos FC' : 'Santos',
    'Sport Recife' : 'Sport',
    'AD São Caetano' : 'São Caetano',
    'AA Ponte Preta' : 'Ponte Preta',
    'Fortaleza EC' : 'Fortaleza',
    'Santa Cruz FC' : 'Santa Cruz',
    'Ipatinga FC' : 'Ipatinga', 'Ipatinga Futebol Clube (MG)' : 'Ipatinga',
    'Associação Portuguesa de Desportos (SP)' : 'Portuguesa',
    'EC Vitória' : 'Vitória',
    'Coritiba FC' : 'Coritiba',
    'Esporte Clube Santo André (SP)' : 'Santo André',
    'Grêmio Prudente Futebol Ltda' : 'Prudente',
    'Avaí FC (SC)' : 'Avaí',
    'Atlético Goianiense' : 'Atlético-GO',
    'Guarani FC' : 'Guarani',
    'Ceará SC' : 'Ceará',
    'EC Bahia' : 'Bahia',
    'América Mineiro' : 'América-MG',
    'Criciúma EC' : 'Criciúma',
    'Joinville EC' : 'Joinville',
    'Cuiabá EC' : 'Cuiabá',
    'RB Bragantino' : 'Bragantino',
    'Figueirense FC' : 'Figueirense'
}

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36'}

In [1062]:
for ano in range(2005,2021,1):
    url = 'https://www.transfermarkt.com.br/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id='+ str(ano)

    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        html = response.read()
        #print(html)
    except HTTPError as e:
        print(e.status, e.reason)
    except URLError as e:
        print(e.reason)
    html = html.decode('utf-8')

    html = trata_html(html)

    soup = BeautifulSoup(html, 'html.parser')

    #print(soup.prettify())

    lista = []
    for item in soup.findAll(['td', {'class': 'hauptlink no-border-links'}, 'td', {'class': 'zentriert'}], limit = 158):
        if item.a != None:
            lista.append(item.a.getText())
        else:
            lista.append(item.getText())


    #print(lista[19:])


    limpeza_lista = filter(lambda val: val != '', lista[19:])
    limpeza_lista = list(limpeza_lista)

    dicionario = gera_dicionario(limpeza_lista ,ano+1)

    classification = {'Time' : [], 'Posição' : []}

    for item in soup.findAll('td', {'class': 'rechts hauptlink'}):
        classification['Posição'].append(item.getText().split(' ')[0])


    for item in soup.findAll('td', {'class': 'no-border-links hauptlink'}):
        classification['Time'].append(item.a.getText())

    while len(classification['Time']) > 20:
        classification['Time'].remove(classification['Time'][0])

    while len(classification['Posição']) > 20:
        classification['Posição'].remove(classification['Posição'][0])

    #classification

    data = pd.DataFrame(classification)
    data2 = pd.DataFrame(dicionario)

    data2.replace(adaptacao, inplace = True)

    data.replace(adaptacao, inplace = True)

    final_data = data.merge(data2, on='Time')

In [1063]:
final_data

,Time,Posição,Jogadores,Média de Idade,Estrangeiros,Média de valor de mercado,Valor de mercado total,Ano
0,Flamengo,1,62,"21,4",6,"1,70 mi. €","105,10 mi. €",2019
1,Internacional,2,44,"23,5",9,"1,53 mi. €","67,25 mi. €",2019
2,Atlético-MG,3,60,"22,7",11,"1,17 mi. €","70,15 mi. €",2019
3,São Paulo,4,47,"22,3",6,"1,07 mi. €","50,20 mi. €",2019
4,Fluminense,5,54,"22,5",3,510 mil €,"27,53 mi. €",2019
5,Grêmio,6,50,"24,2",5,"1,24 mi. €","61,80 mi. €",2019
6,Palmeiras,7,57,"21,9",8,"1,54 mi. €","88,00 mi. €",2019
7,Santos,8,55,"21,9",6,738 mil €,"40,58 mi. €",2019
8,Athletico-PR,9,62,"22,0",4,495 mil €,"30,70 mi. €",2019
9,Bragantino,10,42,"21,8",5,368 mil €,"15,48 mi. €",2019


In [1065]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="myusername",
  password="mypassword"
)

print(mydb)


DatabaseError: 2003 (HY000): Can't connect to MySQL server on 'localhost:3306' (10061)